In [ ]:
using MEDYAN
using StaticArrays
using LinearAlgebra
using Random
using SmallZarrGroups
using CairoMakie
using IterativeSolvers
using FFTW
using DSP
using Plots
#Random.seed!(1234);

In [ ]:
"""
Define the 1D radially dependent Peskin kernel component.
"""
function ϕ(r)
    if r <= -2
        return 0
    elseif -2 <= r < -1
        return inv(8)*(5 + 2*r - sqrt(-7 - 12*r - 4*r^2))
    elseif -1 <= r < 0
        return inv(8)*(3 + 2*r + sqrt(1 - 4*r - 4*r^2)) 
    elseif 0 <= r < 1
        return inv(8)*(3 - 2*r + sqrt(1 + 4*r - 4*r^2)) 
    elseif 1 <= r < 2
        return inv(8)*(5 - 2*r - sqrt(-7 + 12*r - 4*r^2)) 
    elseif 2 <= r
        return 0
    end
end

"""
Define the 3D radially dependent Peskin kernel.
"""
function δₐ(rvec,a)
    return inv(a^3)*ϕ(inv(a)*rvec[1])*ϕ(inv(a)*rvec[2])*ϕ(inv(a)*rvec[3])
end

"""
Fourier transform of nabla vector operator
"""
function gk(kvec,N)
    return [sin(2π*kvec[1]/N[1]),sin(2π*kvec[2]/N[2]),sin(2π*kvec[3]/N[3])]./Δx
end

"""
Fourier transform of laplacian scalar operator.
"""
function αk(kvec,N,μ,ρ,Δx)
    return (2*μ*inv(ρ*Δx^2))*((1-cos(2π*kvec[1]/N[1]))+(1-cos(2π*kvec[2]/N[2]))+(1-cos(2π*kvec[3]/N[3])))
end

"""
Fourier transform of diffusion constant.
"""
function Dk(kvec,N,T,ρ,L,μ,Δx)
    kB = 8.316*1E3 #nm^2*amu/ns^2*K
    if (kvec[1] ∈ [0,N[1]/2]) && (kvec[2] ∈ [0,N[2]/2]) && (kvec[3] ∈ [0,N[3]/2]) 
        return (kB*T*inv(ρ*L^3))*αk(kvec,N,μ,ρ,Δx)
    else
        return (kB*T*inv(2*ρ*L^3))*αk(kvec,N,μ,ρ,Δx)
    end
end

"""
Define the grid of fluid field points in position space.
"""
function pos_grid(Δx)
    
end


"""
Returns force density at point in space from all the elementary particles.
NOTE: need to call MEDYAN functions for node forces, position
NOTE: could make a mutation that mutates the entire force density field
"""
#function force_dens(rgrid,a)   
#end

In [ ]:
#fluid domain parameters
T = 300 #K
L = 1000 #nm
μ = 6.02*1E5 #amu/(nm ns)
ρ = 602 #amu/nm^3
nx = 32
ny = 32
nz = 32

#grid cell edge width
Δx = L/nx 

#Define 3D regular grid for the fluid domain
fluid_grid = MEDYAN.CubicGrid([nx,ny,nz], Δx)
#prod(fluid_grid.n.*fluid_grid.compartmentsize)

In [ ]:
#time parameters
ntstep = 1000
dt = 0.1
#define the grid
N = (100,100,100)
u = zeros(N[1], N[2], N[3])
# include square of high concentration to observe diffusion clearly
u[40:50,40:50,40:50] .= 2

#define fourier transform of discrete, 3D laplacian
k1 = fftfreq(N[1])
k2 = fftfreq(N[2])
k3 = fftfreq(N[3])
tmp = -(k1.^2 + k2.^2 + k3.^2)
c = zeros(length(tmp),length(tmp),length(tmp))
map(i -> c[i,:,:] = ((tmp*transpose(tmp))[i,:])*transpose(tmp), 1:length(tmp))
lap_mat = cbrt.(c)

#divergence operator in terms of FTs
function lap_fft(x)
    return abs.(ifftshift(ifft(fftshift(ifftshift(fft(fftshift(x))).*lap_mat))))
end

#ode stepper or Implicit-Explicit solver
for i in 1:ntstep
    u+=lap_fft(u)*dt
end

#plot state at fixed z-value
Plots.heatmap(u[:,:,45])

In [ ]:
t0 = 0              # Start time 
fs = 44100          # Sampling rate (Hz)
tmax = 0.1*5          # End time       

t = t0:1/fs:tmax;   
signal = sin.(2π * 20 .* t) + sin.(2π * 30 .* t)

F = fftshift(fft(signal))
freqs = fftshift(fftfreq(length(t), fs))

# plots 
time_domain = Plots.plot(t, signal, title = "Signal", label='f',legend=:top)
freq_domain = Plots.plot(freqs, abs.(F), title = "Spectrum", xlim=(-100, +100), xticks=-100:20:100, label="abs.(F)",legend=:top) 
Plots.plot(time_domain, freq_domain, layout = (2,1))

In [ ]:
r = collect(range(0.0,5.0,step=0.01))
f = map(x -> ϕ(x), r)
CairoMakie.lines(r,f,color = :red,linewidth=1)